# Tutorial II: Solving unconstrained optimal control problems using a All-at-once/One-shot method  

Before we begin let

$$
% \DeclareMathOperator{\Div}{div}
% \DeclareMathOperator{\Grad}{grad}
% \DeclareMathOperator{\Curl}{curl}
% \DeclareMathOperator{\Rot}{rot}
% \DeclareMathOperator{\ord}{ord}
% \DeclareMathOperator{\Kern}{ker}
% \DeclareMathOperator{\Image}{im}
% \DeclareMathOperator{\spann}{span}
% \DeclareMathOperator{\dist}{dist}
% \DeclareMathOperator{\diam}{diam}
% \DeclareMathOperator{\sig}{sig}
\newcommand{\RR}{\mathbb{R}}
\newcommand{\NN}{\mathbb{N}}
\newcommand{\VV}{\mathbb{V}}
\newcommand{\dGamma}{\,\mathrm{d} \Gamma}
\newcommand{\dGammah}{\,\mathrm{d} \Gamma_h}
\newcommand{\dx}{\,\mathrm{d}x}
\newcommand{\dy}{\,\mathrm{d}y}
\newcommand{\ds}{\,\mathrm{d}s}
\newcommand{\dt}{\,\mathrm{d}t}
\newcommand{\dS}{\,\mathrm{d}S}
\newcommand{\dV}{\,\mathrm{d}V}
\newcommand{\dX}{\,\mathrm{d}X}
\newcommand{\dY}{\,\mathrm{d}Y}
\newcommand{\dE}{\,\mathrm{d}E}
\newcommand{\dK}{\,\mathrm{d}K}
\newcommand{\dM}{\,\mathrm{d}M}
\newcommand{\cd}{\mathrm{cd}}
\newcommand{\onehalf}{\frac{1}{2}}
\newcommand{\bfP}{\boldsymbol P}
\newcommand{\bfx}{\boldsymbol x}
\newcommand{\bfa}{\boldsymbol a}
\newcommand{\bfu}{\boldsymbol u}
\newcommand{\bfv}{\boldsymbol v}
\newcommand{\bfe}{\boldsymbol e}
\newcommand{\bfg}{\boldsymbol g}
\newcommand{\bfb}{\boldsymbol b}
\newcommand{\bff}{\boldsymbol f}
\newcommand{\bfp}{\boldsymbol p}
\newcommand{\bft}{\boldsymbol t}
\newcommand{\bfj}{\boldsymbol j}
\newcommand{\bfB}{\boldsymbol B}
\newcommand{\bfV}{\boldsymbol V}
\newcommand{\bfE}{\boldsymbol E}
\newcommand{\bfK}{\boldsymbol K}
\newcommand{\mcT}{\mathcal{T}}
\newcommand{\mcL}{\mathcal{L}}
\newcommand{\mcU}{\mathcal{U}}
\newcommand{\ubar}{\overline{u}}
\newcommand{\ybar}{\overline{y}}
\newcommand{\pbar}{\overline{p}}
$$




In this notebook we look again at the distributed optimal control problem

\begin{align*}
J(y, u) = \dfrac{1}{2} \| y - y_{\Omega}\|_{\Omega}^2 + \dfrac{\gamma}{2} \| u \|^2_{\Omega} \to \min
\end{align*}

subject to state equation
\begin{alignat}{2}
-\Delta y &= f + \beta u & &\quad \text{in } \Omega,
\\
        y &= 0 & & \quad \text{on } \Gamma = \partial \Omega, 
\end{alignat}
and $u \in \mathcal{U}_{ad}$  for some convex subset of $\mathcal{U} = L^2(\Omega)$. Here,
$\beta $ is simply some positive constant, and for simplicity, we pick $\Omega = (0,1)^2 \subset \mathbb{R}^2$. 

Finally, our target function is 
$$y_{\Omega} = 10x_1(1-x_1)x_2(1-x_2).
$$.
This example is take from [ManzoniQuarteroniSalsa2021, Section 6.5.1, Test case 1](https://link.springer.com/10.1007/978-3-030-77226-0).



This time we want to solve the problem by solving the state, co-state problem and the optimality problem all at once. 
We start from the optimality system
\begin{align*}
a(\ybar, \varphi)  &= (f + \beta \ubar, \varphi)_{\Omega}  \quad \forall \varphi \in V
\\
a(\psi, \pbar) &= (\ybar - y_{\Omega}, \psi)_{\Omega}  \quad \forall \psi \in V 
\\
(\gamma \ubar + \beta \pbar, v)_{\Omega} &= 0 \quad \forall v \in \mathcal{U}.
\end{align*} 

We can simply move all unknowns to the left-hand side to see that we need to find
$(\ybar, \pbar, \ubar) \in V\times V\times U$ such that $\forall\; (\phi, \psi, v) \in V \times V \times U$  

\begin{align*}
a(\ybar, \varphi) - (\beta \ubar, \varphi)_{\Omega}  &= (f, \varphi)_{\Omega} 
\\
a(\psi, \pbar) - (\ybar, \psi)_{\Omega}&= -(y_{\Omega}, \psi)_{\Omega} 
\\
(\gamma \ubar + \beta \pbar, v)_{\Omega} &= 0
\end{align*} 

Before we start implementing a solver for this system, let us activate our Julia project environment first. 

In [2]:
import Pkg
Pkg.activate("../")
Pkg.status()

Status `~/Repositories/tma4183-optimization-ii/Project.toml`
⌅ [13f3f980] CairoMakie v0.8.13
  [5789e2e9] FileIO v1.16.0
⌅ [e9467ef8] GLMakie v0.6.13
  [5c1252a2] GeometryBasics v0.4.5
  [705231aa] Gmsh v0.2.2
  [56d4f2e9] Gridap v0.17.17
  [3025c34a] GridapGmsh v0.6.1
  [41f30b06] GridapMakie v0.1.2
  [55e38337] GridapODEs v0.8.0
  [98b081ad] Literate v2.14.0
⌅ [ee78f7c6] Makie v0.17.13
  [eacbb407] Meshes v0.28.0
  [f0f68f2c] PlotlyJS v0.18.10
⌃ [91a5bcdd] Plots v1.32.0
⌃ [276b4fcb] WGLMakie v0.6.13
Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ cannot be upgraded. To see why use `status --outdated`


  Activating project at `~/Repositories/tma4183-optimization-ii`


## Problem 1

 * Implement a finite element solver for complete optimality system using the following Gridap code snippets.
 * Compare the resulting solution with the solution you obtained from Tutorial I where you where asked to implement
a Steepest Descent method for the same OCP.

First we need to define a mixed finite element space representing the Cartesian product of function spaces $V \times V \times U$.
As usual we start with the test function space first. We discretize the state, co-state and optimality problem using the same finite element spaces,
with the important distinction that $V$ in contrast to $U$ needs to incorporate boundary conditions as well.  

```julia
# Define your grid
model = ...

order = 1
V_ref = ReferenceFE(lagrangian, Float64, order)
U_ref = V_ref

# Define individual spaces
Vy = TestFESpace(model, V_ref, conformity=:H1, dirichlet_tags="boundary")
Vp = TestFESpace(model, V_ref, conformity=:H1, dirichlet_tags="boundary")
Vu = TestFESpace(model, V_ref, conformity=:H1)

# Build the combined space
X = MultiFieldFESpace([Vy, Vp, Vu])
```

Next, we need to define the corresponding trial function spaces.
```julia

# Define boundary conditions for y and p
y_D(x) = 0
p_D(x) = 0

Uy = TrialFESpace(Vy, y_D)
Up = TrialFESpace(Vp, p_D)
Uu = TrialFESpace(Vu)

Y = MultiFieldFESpace(Uy, Up, Uu)
```

You define you triangulations and measures as usual.
Later, when you define your bilinear and linear forms, you define it for the **total** system, i.e. you combine
all individual bilinear forms arising in our system into one single bilinear form:
```julia
A((y,p,u), (φ, ψ, v)) = ...
L((φ, ψ, v)) = ...
```

Finally, you solve your system as usual.
```julia
op = AffineFEOperator(A,L,X,Y)
yh, ph, uh = solve(op)

writevtk(Ω,"results",order=2,cellfields=["uh"=>uh,"ph"=>ph, "uh"=>uh])
```

DONE! Now have fun coding this up.

In [14]:
using Gridap
using Gridap.Algebra

γ = 1e-4
β = 1
f = 0
domain = (0,1,0,1)
partition = (50, 50)
model = CartesianDiscreteModel(domain, partition) |> simplexify
writevtk(model, "mesh_nx$(partition[1])_ny$(partition[2])")

# Define reference function spaces
order = 1 # piecewise constant functions
V_ref = ReferenceFE(lagrangian, Float64, order)
U_ref = V_ref

# Define individual spaces
Vy = TestFESpace(model, V_ref, conformity=:H1, dirichlet_tags="boundary")
Vp = TestFESpace(model, V_ref, conformity=:H1, dirichlet_tags="boundary")
Vu = TestFESpace(model, V_ref, conformity=:H1)

# Build the combined space
X = MultiFieldFESpace([Vy, Vp, Vu])

# Set up measures
degree = 2*order
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

# Define boundary conditions for y and p
y_D(x) = 0
p_D(x) = 0

yΩ(x) = 10*x[1]*(1-x[1])*x[2]*(1-x[2]) # Target function

Uy = TrialFESpace(Vy, y_D)
Up = TrialFESpace(Vp, p_D)
Uu = TrialFESpace(Vu)

Y = MultiFieldFESpace([Uy, Up, Uu])

A((y, p, u), (φ, ψ, v)) = ∫(∇(y)⋅∇(φ))*dΩ + ∫(∇(p)⋅∇(ψ))*dΩ + ∫((γ*u+β*p)*(v))*dΩ - ∫((y)*(ψ))*dΩ - ∫((β*u)*(φ))*dΩ
L((φ, ψ, v)) = ∫((f)*(φ))*dΩ - ∫((yΩ)*(ψ))*dΩ

# Solve linear system
op = AffineFEOperator(A,L,X,Y)
yh, ph, uh = solve(op)

writevtk(Ω,"results",order=2,cellfields=["uh"=>uh,"ph"=>ph, "uh"=>uh])

(["results.vtu"],)